# 1. Implementation Details
The Rasch Model implementation in eduTech is based entirely on the implementation described in
David Barber's book *"Bayesian Reasoning and Machine Learning."* Barber describes the processes of estimating 
the model parameters in terms of Maximizing Likelihood, thus the current implementation in this package
follows the same convention. The details of which are described below.

Probability of a correct answer: 

$$p(X_{ij} = 1| s_i, q_j) = \sigma(s_i - q_j)$$

Loss Function:

$$\sum_{i,j} X_{ij}\log(\sigma(s_i-q_j) + (1-X_{ij})\log(1-\sigma(s_i-q_j))$$

Derivative of loss in terms of student abilities:

$$\frac{\partial{loss}}{\partial{s_i}}  = \sum_j X_{ij} - \log(\sigma(s_i-q_j) $$

Derivative of loss in terms of question difficulties:

$$\frac{\partial{loss}}{\partial{q_j}}  = -\sum_i X_{ij} - \log(\sigma(s_i-q_j) $$

In practice these gradients are normalized by dividing by the gradient vector's mean as well as applying a learning rate to them.

# 2. Test Case
The verification of eduTech's implementation comes from a matlab project that implements the
same processes that Berber describes in his book.
The repository can be found here: https://github.com/cosmicBboy/bayesian-reasoning-machine-learning. 
The testing data comes from a Penn state tutorial on item response theory (irt) which is the field of study 
that the Rasch model comes from. This data can be found here: https://quantdev.ssri.psu.edu/tutorials/introduction-irt-modeling.

## 2.1 How to test for yourself
First you are going to need to load the testing data into the matlab program. This is easily achieved by first converting the file 
into a .txt and then using matlab's importdata function.

```
testData = importData("ouirt.txt")
``` 

Secondly, you are going to need to transpose this dataset. This is simply how the matlab implementation expects it. 
```
testData = transpose(testData); 
```

Third, you are going to want to create three files named as below:
* rasch.m
* sigma.m
* mynansum.m

And copy and paste the following code snippets (taken from the repository mentioned above) into these files respectively:
* rasch.m

```
function [a d ll]=rasch(X,opts)
%rasch Fit a Rasch model to a binary matrix X
% X contains binary data, coded 0 or 1 (nan if missing)
% opts.eta_a - ability learning rate
% opts.eta_d - difficulty learning rate
% opts.plotprogress
% opts.tol - log likelihood termination tolerance
% see demoRasch.m

% Maximum Likelood training using simple Gradient ascent:
[Q S]=size(X);
a=zeros(1,S); d=zeros(Q,1);

if ~isfield(opts,'eta_a'); opts.eta_a=1; end % alpha learning rate
if ~isfield(opts,'eta_d'); opts.eta_d=1; end % delta learning rate
if ~isfield(opts,'plotprogress') opts.plotprogress=0; end % delta learning rate
if ~isfield(opts,'tol') opts.tol=1e-6; end % termination toleration

ll_old=-1e10;
for loop=1:opts.maxits
    sig = sigma(repmat(a,Q,1) - repmat(d,1,S));
    loglik(loop) = mynansum(mynansum(X.*log(sig)+(1-X).*log(1-sig)));
    
    grada = mynansum(X-sig,1);
    gradd = -mynansum(X-sig,2);
    
    a = a + opts.eta_a*grada/S;
    d = d + opts.eta_d*gradd/Q;
    if abs(loglik(end)-ll_old)<opts.tol; break; end; ll_old=loglik(end);
    
    if opts.plotprogress; plot(loglik); title('log likelihood'); drawnow; end
end
```

* sigma.m

```
function s=sigma(x)
%SIGMA 1./(1+exp(-x))
% s=sigma(x) = 1./(1+exp(-x))
s=1./(1+exp(-x));
```

* mynansum.m

```
function y = mynansum(x,d)
%MYNANSUM sum of values that are not nan
x(isnan(x))=0;
if nargin==1; y = sum(x);
else y = sum(x,d);
end
```


Then to setup the options that correspond to this test run the following:

```
opts.maxits = 100
opts.plotprogress = 1
opts.eta_a = .01
opts.eta_d = .01
```

Finally, run the following:

```
[student_abilities, question_difficulties] = rasch(testData, opts) 
```

The following question difficulties should be obtained (student abilities has been omitted due to the overwhelming
size): 

```
1.7328
1.0044
0.7626
0.8660
0.2489
0.7811
0.3553
0.6898
0.7626
2.5463
```



Now that we have the model's results in the matlab version, we compare to eduTech's implementation




In [21]:
# This will eventually change to edutech.irt but for now it's a relative import
import pandas as pd 
from irt.rasch import RaschModel

def load_test_data():
    with open("ouirt.txt", "r") as f:
        rows = []
        for line in f.readlines():
            current_row = []
            for no in line.split(" "):
                try:
                    current_row.append(int(no))
                except ValueError:
                    continue
            rows.append(current_row)
        return pd.DataFrame(rows)



test_model = RaschModel(learning_rate=.01)
test_data = load_test_data()
test_model.fit(test_data, epochs=100)
student_abilities, question_difficulties = test_model.get_model_descriptors()
print("======= Student Abilities =======")
print(student_abilities[0:10])
print(".....")
print("======= Question Difficulties =======")
print(question_difficulties)


LOSS AFTER 0 ITERATIONS: -3465.7294921875
Finished fitting with a final loss of -2863.20263671875
======= Student Abilities =======
[[ 0.00975408]
 [-0.00022631]
 [ 0.00176978]
 [-0.00621456]
 [-0.00421847]
 [-0.00421847]
 [ 0.00176978]
 [-0.00222239]
 [-0.00222239]
 [-0.00222239]]
.....
======= Question Difficulties =======
[[1.732794   1.0043824  0.7625858  0.8660217  0.24889438 0.7810914
  0.35531518 0.6897581  0.7625858  2.5463223 ]]


## 2.2 Results
The same question difficulties are obtained, and through careful examination the same student abilities are obtained 
(with margin for error in rounding). This indicates that the Rasch model implemented in eduTech is reliable. For more information
on why Maximum Likelihood was chosen as the optimizer as opposed to Marginal Maximum Likelihood or Bayesian Estimation  
see the introduction notebook at the top of this module. 

